In [1]:
import os
import gc
import math

# numpy
import numpy as np
import numpy.ma as ma

from jax import vmap, grad, jit, random
from jax.config import config
# from jax.experimental import optimizers
import jax.numpy as jnp
from jax.tree_util import tree_map, tree_flatten, tree_unflatten, tree_leaves

config.update("jax_enable_x64", True)

# from skimage.restoration import estimate_sigma
from skimage.transform import resize

### CRPS
import properscoring as ps

### SSIM and PSNR
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import filters

# plotting
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('default')

from matplotlib.gridspec import GridSpec
from matplotlib import colors
from cartopy import config
import cartopy.crs as ccrs

# netCDF
import netCDF4 as nc

from tqdm.notebook import tqdm
from tqdm.notebook import trange

import pickle

In [2]:
from jax.scipy.special import ndtr as ndtr

def normal_cdf(x, m, s):
    x = (x - m) / s
    return np.array(ndtr(x))

def pit_image(y, yhat, ysig):
    score = normal_cdf(y, yhat, ysig)
    emp_hist = np.histogram(score.flatten(), density=True)[0]
    return np.sqrt(np.mean((emp_hist - 1)**2))

# T2M UQ

In [24]:
xhist = pickle.load(open('../data/saved/xhist_tas.pkl', 'rb'))
xrcp = pickle.load(open('../data/saved/xrcp_tas.pkl', 'rb'))

nval = 72
nmod = len(xhist)

In [17]:
sgpr = [np.load(f'tas_predictions/sgpr_tas_{m}.npz.npy') for m in trange(nmod)]
sgpr_var = [np.load(f'tas_predictions/sgpr_tas_{m}_var.npz.npy') for m in trange(nmod)]

xreg = [np.load(f'tas_predictions/xreg_tas_{m}.npz.npy') for m in trange(nmod)]
xreg_var = [np.load(f'tas_predictions/xreg_tas_{m}_var.npz.npy') for m in trange(nmod)]

xbar = [np.load(f'tas_predictions/xbar_tas_{m}.npz.npy') for m in trange(nmod)]
xbar_var = [np.load(f'tas_predictions/xbar_tas_{m}_var.npz.npy') for m in trange(nmod)]

xwea = [np.load(f'tas_predictions/xwea_tas_{m}.npz.npy') for m in trange(nmod)]
xwea_var = [np.load(f'tas_predictions/xbar_tas_{m}_var.npz.npy') for m in trange(nmod)]

gpse = [np.load(f'tas_predictions/gpr_exp_tas_{m}.npz.npy') for m in trange(nmod)]
gpse_var = [np.load(f'tas_predictions/gpr_exp_tas_var_{m}.npz.npy') for m in trange(nmod)]

gpex = [np.load(f'tas_predictions/gpr_se_tas_{m}.npz.npy') for m in trange(nmod)]
gpex_var = [np.load(f'tas_predictions/gpr_se_tas_var_{m}.npz.npy') for m in trange(nmod)]

delt = [np.load(f'tas_predictions/del_tas_{m}.npz.npy') for m in trange(nmod)]
delt_var = [np.load(f'tas_predictions/del_tas_{m}_var.npz.npy') for m in trange(nmod)]


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [42]:
ntest = sgpr_tas[0].shape[0] + nval
months = 12 * 10
decades = int((ntest-nval)/(months))

In [45]:
sgpr_crps = np.zeros((decades, nmod))
xreg_crps = np.zeros((decades, nmod))
xwea_crps = np.zeros((decades, nmod))
xbar_crps = np.zeros((decades, nmod))
gpse_crps = np.zeros((decades, nmod))
gpex_crps = np.zeros((decades, nmod))
delt_crps = np.zeros((decades, nmod))

sgpr_pit = np.zeros((decades, nmod))
xreg_pit = np.zeros((decades, nmod))
xwea_pit = np.zeros((decades, nmod))
xbar_pit = np.zeros((decades, nmod))
gpse_pit = np.zeros((decades, nmod))
gpex_pit = np.zeros((decades, nmod))
delt_pit = np.zeros((decades, nmod))

for m1 in trange(nmod):
    ytest = np.array(xrcp[m1])[nval:ntest]
    nlat = ytest.shape[1]
    scale = np.cos(np.linspace(math.pi/2, -math.pi/2, nlat))
    
    for d in range(decades):
        
        ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
        sgpr_dec = sgpr[m1][(months*d):(months*(d+1))].astype('float32')
        xreg_dec = xreg[m1][(months*d):(months*(d+1))].astype('float32')
        xwea_dec = xwea[m1][(months*d):(months*(d+1))].astype('float32')
        xbar_dec = xbar[m1][(months*d):(months*(d+1))].astype('float32')
        gpse_dec = gpse[m1][(months*d):(months*(d+1))].astype('float32')
        gpex_dec = gpex[m1][(months*d):(months*(d+1))].astype('float32')
        delt_dec = delt[m1][(months*d):(months*(d+1))].astype('float32')
        
        ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
        sgpr_var_dec = sgpr_var[m1][(months*d):(months*(d+1))].astype('float32')
        xreg_var_dec = xreg_var[m1][(months*d):(months*(d+1))].astype('float32')
        xwea_var_dec = xwea_var[m1][(months*d):(months*(d+1))].astype('float32')
        xbar_var_dec = xbar_var[m1][(months*d):(months*(d+1))].astype('float32')
        gpse_var_dec = gpse_var[m1][(months*d):(months*(d+1))].astype('float32')
        gpex_var_dec = gpex_var[m1][(months*d):(months*(d+1))].astype('float32')
        delt_var_dec = delt_var[m1][(months*d):(months*(d+1))].astype('float32')
        
        ### crps
        sgpr_crps_dec = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
        sgpr_crps_dec = np.mean(sgpr_crps_dec * scale[None,:,None])
        
        xreg_crps_dec = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
        xreg_crps_dec = np.mean(xreg_crps_dec * scale[None,:,None])
        
        xbar_crps_dec = ps.crps_gaussian(ytest_dec, xbar_dec, xbar_var_dec)
        xbar_crps_dec = np.mean(xbar_crps_dec * scale[None,:,None])
        
        xwea_crps_dec = ps.crps_gaussian(ytest_dec, xwea_dec, xwea_var_dec)
        xwea_crps_dec = np.mean(xbar_crps_dec * scale[None,:,None])
        
        gpse_crps_dec = ps.crps_gaussian(ytest_dec, gpse_dec, gpse_var_dec[:,None,None])
        gpse_crps_dec = np.mean(gpse_crps_dec * scale[None,:,None])
        
        gpex_crps_dec = ps.crps_gaussian(ytest_dec, gpex_dec, gpex_var_dec[:,None,None])
        gpex_crps_dec = np.mean(gpex_crps_dec * scale[None,:,None])
        
        delt_crps_dec = ps.crps_gaussian(delt_dec, delt_dec, delt_var_dec)
        delt_crps_dec = np.mean(delt_crps_dec * scale[None,:,None])
        
        sgpr_crps[d, m1] = sgpr_crps_dec
        xreg_crps[d, m1] = xreg_crps_dec
        xwea_crps[d, m1] = xwea_crps_dec
        xbar_crps[d, m1] = xbar_crps_dec
        gpse_crps[d, m1] = gpse_crps_dec
        gpex_crps[d, m1] = gpex_crps_dec
        delt_crps[d, m1] = delt_crps_dec
        
        ## PIT
        sgpr_pit[d, m1] = np.mean([pit_image(ytest_dec[i], sgpr_dec[i], sgpr_var_dec[i]) for i in trange(months, leave = False)])
        xreg_pit[d, m1] = np.mean([pit_image(ytest_dec[i], xreg_dec[i], xreg_var_dec[i]) for i in trange(months, leave = False)])
        xwea_pit[d, m1] = np.mean([pit_image(ytest_dec[i], xwea_dec[i], xwea_var_dec[i]) for i in trange(months, leave = False)])
        xbar_pit[d, m1] = np.mean([pit_image(ytest_dec[i], xbar_dec[i], xbar_var_dec[i]) for i in trange(months, leave = False)])
        gpse_pit[d, m1] = np.mean([pit_image(ytest_dec[i], gpse_dec[i], gpse_var_dec[i]) for i in trange(months, leave = False)])
        gpex_pit[d, m1] = np.mean([pit_image(ytest_dec[i], gpex_dec[i], gpex_var_dec[i]) for i in trange(months, leave = False)])
        delt_pit[d, m1] = np.mean([pit_image(ytest_dec[i], delt_dec[i], delt_var_dec[i]) for i in trange(months, leave = False)])

        
#         ### coverage
#         sgpr_lb = sgpr_dec - 1.96 * sgpr_var_dec[:,None,None]
#         sgpr_ub = sgpr_dec + 1.96 * sgpr_var_dec[:,None,None]
#         sgpr_oob = (ytest_dec > sgpr_ub) + (ytest_dec < sgpr_lb)
#         sgpr_oob = sgpr_oob * scale[None,:,None]
        
#         xreg_lb = xreg_dec - 1.96 * xreg_var_dec
#         xreg_ub = xreg_dec + 1.96 * xreg_var_dec
#         xreg_oob = (ytest_dec > xreg_ub) + (ytest_dec < xreg_lb)
#         xreg_oob = xreg_oob * scale[None,:,None]
        
#         xbar_lb = xbar_dec - 1.96 * xbar_var_dec
#         xbar_ub = xbar_dec + 1.96 * xbar_var_dec
#         xbar_oob = (ytest_dec > xbar_ub) + (ytest_dec < xbar_lb)
#         xbar_oob = xbar_oob * scale[None,:,None]
        
#         xwea_lb = xwea_dec - 1.96 * xwea_var_dec
#         xwea_ub = xwea_dec + 1.96 * xwea_var_dec
#         xwea_oob = (ytest_dec > xwea_ub) + (ytest_dec < xwea_lb)
#         xwea_oob = xwea_oob * scale[None,:,None]
        
#         sgpr_cover[d, m1] = 1 - np.mean(sgpr_oob)
#         xreg_cover[d, m1] = 1 - np.mean(xreg_oob)
#         xbar_cover[d, m1] = 1 - np.mean(xbar_oob)
#         xwea_cover[d, m1] = 1 - np.mean(xwea_oob)
        
    

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

...

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

In [48]:
from tabulate import tabulate

tas_metrics = np.array([np.arange(1, 9),
          np.mean(sgpr_crps, axis = 1),
          np.mean(xreg_crps, axis = 1),
          np.mean(xwea_crps, axis = 1),
          np.mean(xbar_crps, axis = 1),
          np.mean(gpse_crps, axis = 1),
          np.mean(gpex_crps, axis = 1),
          np.mean(delt_crps, axis = 1),
          np.mean(sgpr_pit, axis = 1),
          np.mean(xreg_pit, axis = 1),
          np.mean(xwea_pit, axis = 1),
          np.mean(xbar_pit, axis = 1),
          np.mean(gpse_pit, axis = 1),
          np.mean(gpex_pit, axis = 1),
          np.mean(delt_pit, axis = 1)]).T

print(tabulate(np.round(tas_metrics, 2),
               numalign = 'center',
               headers = ['NN-GPR', 'REG', 'WEA', 'EA', 'GPSE', 'GPEX', 'DELT'] * 2,
               tablefmt = "latex_booktabs"))

\begin{tabular}{ccccccccccccccc}
\toprule
    &  NN-GPR  &  REG  &  WEA  &  EA  &  GPSE  &  GPEX  &  DELT  &  NN-GPR  &  REG  &  WEA  &  EA  &  GPSE  &  GPEX  &  DELT  \\
\midrule
 1  &   0.46   & 0.43  & 0.46  & 0.73 &  0.46  &  0.46  &  2.44  &   0.58   & 0.27  &  0.4  & 0.47 &  0.58  &  0.56  &  1.58  \\
 2  &   0.47   & 0.43  & 0.46  & 0.72 &  0.47  &  0.47  &  2.49  &   0.58   & 0.28  &  0.4  & 0.46 &  0.56  &  0.53  &  1.58  \\
 3  &   0.48   & 0.44  & 0.46  & 0.73 &  0.49  &  0.5   &  2.53  &   0.58   & 0.33  & 0.43  & 0.5  &  0.54  &  0.51  &  1.58  \\
 4  &   0.5    & 0.46  & 0.47  & 0.74 &  0.52  &  0.53  &  2.57  &   0.56   & 0.37  & 0.44  & 0.5  &  0.51  &  0.47  &  1.59  \\
 5  &   0.52   & 0.46  & 0.47  & 0.74 &  0.54  &  0.56  &  2.61  &   0.54   & 0.39  & 0.45  & 0.51 &  0.47  &  0.45  &  1.6   \\
 6  &   0.53   & 0.47  & 0.47  & 0.74 &  0.57  &  0.6   &  2.64  &   0.53   & 0.42  & 0.45  & 0.51 &  0.45  &  0.45  &  1.6   \\
 7  &   0.55   & 0.48  & 0.47  & 0.74 &  0.59 

# PR UQ

In [3]:
xhist = pickle.load(open('../data/saved/xhist_pr.pkl', 'rb'))
xrcp = pickle.load(open('../data/saved/xrcp_pr.pkl', 'rb'))

nval = 72
nmod = len(xhist)

In [4]:
sgpr = [np.load(f'pr_predictions/sgpr_pr_{m}.npz.npy') for m in trange(nmod)]
sgpr_var = [np.load(f'pr_predictions/sgpr_pr_{m}_var.npz.npy') for m in trange(nmod)]

xreg = [np.load(f'pr_predictions/xreg_pr_{m}.npz.npy') for m in trange(nmod)]
xreg_var = [np.load(f'pr_predictions/xreg_pr_{m}_var.npz.npy') for m in trange(nmod)]

xbar = [np.load(f'pr_predictions/xbar_pr_{m}.npz.npy') for m in trange(nmod)]
xbar_var = [np.load(f'pr_predictions/xbar_pr_{m}_var.npz.npy') for m in trange(nmod)]

xwea = [np.load(f'pr_predictions/xwea_pr_{m}.npz.npy') for m in trange(nmod)]
xwea_var = [np.load(f'pr_predictions/xbar_pr_{m}_var.npz.npy') for m in trange(nmod)]

gpse = [np.load(f'pr_predictions/gpr_exp_pr_{m}.npz.npy') for m in trange(nmod)]
gpse_var = [np.load(f'pr_predictions/gpr_exp_pr_var_{m}.npz.npy') for m in trange(nmod)]

gpex = [np.load(f'pr_predictions/gpr_se_pr_{m}.npz.npy') for m in trange(nmod)]
gpex_var = [np.load(f'pr_predictions/gpr_se_pr_var_{m}.npz.npy') for m in trange(nmod)]

delt = [np.load(f'pr_predictions/del_pr_{m}.npz.npy') for m in trange(nmod)]
delt_var = [np.load(f'pr_predictions/del_pr_{m}_var.npz.npy') for m in trange(nmod)]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [35]:
ntest = sgpr[0].shape[0] + nval
months = 12 * 10
decades = int((ntest-nval)/(months) + 1)

In [34]:
12 * 9

108

In [29]:
ytest_dec.shape

(108, 144, 192)

In [30]:
sgpr_dec.shape

(108, 144, 192)

In [37]:
sgpr_crps = np.zeros((decades, nmod))
xreg_crps = np.zeros((decades, nmod))
xwea_crps = np.zeros((decades, nmod))
xbar_crps = np.zeros((decades, nmod))
gpse_crps = np.zeros((decades, nmod))
gpex_crps = np.zeros((decades, nmod))
delt_crps = np.zeros((decades, nmod))

sgpr_pit = np.zeros((decades, nmod))
xreg_pit = np.zeros((decades, nmod))
xwea_pit = np.zeros((decades, nmod))
xbar_pit = np.zeros((decades, nmod))
gpse_pit = np.zeros((decades, nmod))
gpex_pit = np.zeros((decades, nmod))
delt_pit = np.zeros((decades, nmod))

for m1 in trange(nmod):
    ytest = np.array(xrcp[m1])[nval:ntest]
    nlat = ytest.shape[1]
    scale = np.cos(np.linspace(math.pi/2, -math.pi/2, nlat))
    
    for d in range(decades):
        
        if d < 7 :
            months = 12 * 10
        else:
            months = 12 * 9
        
        ytest_dec = ytest[(months*d):(months*(d+1))].astype('float32')
        sgpr_dec = sgpr[m1][(months*d):(months*(d+1))].astype('float32')
        xreg_dec = xreg[m1][(months*d):(months*(d+1))].astype('float32')
        xwea_dec = xwea[m1][(months*d):(months*(d+1))].astype('float32')
        xbar_dec = xbar[m1][(months*d):(months*(d+1))].astype('float32')
        gpse_dec = gpse[m1][(months*d):(months*(d+1))].astype('float32')
        gpex_dec = gpex[m1][(months*d):(months*(d+1))].astype('float32')
        delt_dec = delt[m1][(months*d):(months*(d+1))].astype('float32')
        
        sgpr_var_dec = sgpr_var[m1][(months*d):(months*(d+1))].astype('float32')
        xreg_var_dec = xreg_var[m1][(months*d):(months*(d+1))].astype('float32')
        xwea_var_dec = xwea_var[m1][(months*d):(months*(d+1))].astype('float32')
        xbar_var_dec = xbar_var[m1][(months*d):(months*(d+1))].astype('float32')
        gpse_var_dec = gpse_var[m1][(months*d):(months*(d+1))].astype('float32')
        gpex_var_dec = gpex_var[m1][(months*d):(months*(d+1))].astype('float32')
        delt_var_dec = delt_var[m1][(months*d):(months*(d+1))].astype('float32')
        
        ### crps
        sgpr_crps_dec = ps.crps_gaussian(ytest_dec, sgpr_dec, sgpr_var_dec[:,None,None])
        sgpr_crps_dec = np.mean(sgpr_crps_dec * scale[None,:,None])
        
        xreg_crps_dec = ps.crps_gaussian(ytest_dec, xreg_dec, xreg_var_dec)
        xreg_crps_dec = np.mean(xreg_crps_dec * scale[None,:,None])
        
        xbar_crps_dec = ps.crps_gaussian(ytest_dec, xbar_dec, xbar_var_dec)
        xbar_crps_dec = np.mean(xbar_crps_dec * scale[None,:,None])
        
        xwea_crps_dec = ps.crps_gaussian(ytest_dec, xwea_dec, xwea_var_dec)
        xwea_crps_dec = np.mean(xbar_crps_dec * scale[None,:,None])
        
        gpse_crps_dec = ps.crps_gaussian(ytest_dec, gpse_dec, gpse_var_dec[:,None,None])
        gpse_crps_dec = np.mean(gpse_crps_dec * scale[None,:,None])
        
        gpex_crps_dec = ps.crps_gaussian(ytest_dec, gpex_dec, gpex_var_dec[:,None,None])
        gpex_crps_dec = np.mean(gpex_crps_dec * scale[None,:,None])
        
        delt_crps_dec = ps.crps_gaussian(delt_dec, delt_dec, delt_var_dec)
        delt_crps_dec = np.mean(delt_crps_dec * scale[None,:,None])
        
        sgpr_crps[d, m1] = sgpr_crps_dec
        xreg_crps[d, m1] = xreg_crps_dec
        xwea_crps[d, m1] = xwea_crps_dec
        xbar_crps[d, m1] = xbar_crps_dec
        gpse_crps[d, m1] = gpse_crps_dec
        gpex_crps[d, m1] = gpex_crps_dec
        delt_crps[d, m1] = delt_crps_dec
        
        ## PIT
        sgpr_pit[d, m1] = np.mean([pit_image(ytest_dec[i], sgpr_dec[i], sgpr_var_dec[i]) for i in trange(months, leave = False)])
        xreg_pit[d, m1] = np.mean([pit_image(ytest_dec[i], xreg_dec[i], xreg_var_dec[i]) for i in trange(months, leave = False)])
        xwea_pit[d, m1] = np.mean([pit_image(ytest_dec[i], xwea_dec[i], xwea_var_dec[i]) for i in trange(months, leave = False)])
        xbar_pit[d, m1] = np.mean([pit_image(ytest_dec[i], xbar_dec[i], xbar_var_dec[i]) for i in trange(months, leave = False)])
        gpse_pit[d, m1] = np.mean([pit_image(ytest_dec[i], gpse_dec[i], gpse_var_dec[i]) for i in trange(months, leave = False)])
        gpex_pit[d, m1] = np.mean([pit_image(ytest_dec[i], gpex_dec[i], gpex_var_dec[i]) for i in trange(months, leave = False)])
        delt_pit[d, m1] = np.mean([pit_image(ytest_dec[i], delt_dec[i], delt_var_dec[i]) for i in trange(months, leave = False)])

        
#         ### coverage
#         sgpr_lb = sgpr_dec - 1.96 * sgpr_var_dec[:,None,None]
#         sgpr_ub = sgpr_dec + 1.96 * sgpr_var_dec[:,None,None]
#         sgpr_oob = (ytest_dec > sgpr_ub) + (ytest_dec < sgpr_lb)
#         sgpr_oob = sgpr_oob * scale[None,:,None]
        
#         xreg_lb = xreg_dec - 1.96 * xreg_var_dec
#         xreg_ub = xreg_dec + 1.96 * xreg_var_dec
#         xreg_oob = (ytest_dec > xreg_ub) + (ytest_dec < xreg_lb)
#         xreg_oob = xreg_oob * scale[None,:,None]
        
#         xbar_lb = xbar_dec - 1.96 * xbar_var_dec
#         xbar_ub = xbar_dec + 1.96 * xbar_var_dec
#         xbar_oob = (ytest_dec > xbar_ub) + (ytest_dec < xbar_lb)
#         xbar_oob = xbar_oob * scale[None,:,None]
        
#         xwea_lb = xwea_dec - 1.96 * xwea_var_dec
#         xwea_ub = xwea_dec + 1.96 * xwea_var_dec
#         xwea_oob = (ytest_dec > xwea_ub) + (ytest_dec < xwea_lb)
#         xwea_oob = xwea_oob * scale[None,:,None]
        
#         sgpr_cover[d, m1] = 1 - np.mean(sgpr_oob)
#         xreg_cover[d, m1] = 1 - np.mean(xreg_oob)
#         xbar_cover[d, m1] = 1 - np.mean(xbar_oob)
#         xwea_cover[d, m1] = 1 - np.mean(xwea_oob)
        
    

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

...

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

  0%|          | 0/108 [00:00<?, ?it/s]

In [38]:
from tabulate import tabulate

pr_metrics = np.array([np.arange(1, 9),
          np.mean(sgpr_crps, axis = 1),
          np.mean(xreg_crps, axis = 1),
          np.mean(xwea_crps, axis = 1),
          np.mean(xbar_crps, axis = 1),
          np.mean(gpse_crps, axis = 1),
          np.mean(gpex_crps, axis = 1),
          np.mean(delt_crps, axis = 1),
          np.mean(sgpr_pit, axis = 1),
          np.mean(xreg_pit, axis = 1),
          np.mean(xwea_pit, axis = 1),
          np.mean(xbar_pit, axis = 1),
          np.mean(gpse_pit, axis = 1),
          np.mean(gpex_pit, axis = 1),
          np.mean(delt_pit, axis = 1)]).T

print(tabulate(np.round(pr_metrics, 2),
               numalign = 'center',
               headers = ['NN-GPR', 'REG', 'WEA', 'EA', 'GPSE', 'GPEX', 'DELT'] * 2,
               tablefmt = "latex_booktabs"))

\begin{tabular}{ccccccccccccccc}
\toprule
    &  NN-GPR  &  REG  &  WEA  &  EA  &  GPSE  &  GPEX  &  DELT  &  NN-GPR  &  REG  &  WEA  &  EA  &  GPSE  &  GPEX  &  DELT  \\
\midrule
 1  &   0.58   & 0.56  &  0.4  & 0.63 &  0.58  &  0.58  &  0.42  &   0.78   & 0.26  & 0.34  & 0.36 &  0.76  &  0.75  &  0.59  \\
 2  &   0.59   & 0.57  &  0.4  & 0.64 &  0.58  &  0.59  &  0.42  &   0.77   & 0.26  & 0.33  & 0.36 &  0.76  &  0.75  &  0.59  \\
 3  &   0.59   & 0.58  & 0.41  & 0.65 &  0.59  &  0.59  &  0.42  &   0.76   & 0.26  & 0.33  & 0.36 &  0.76  &  0.75  &  0.59  \\
 4  &   0.61   & 0.59  & 0.41  & 0.65 &  0.6   &  0.61  &  0.42  &   0.74   & 0.25  & 0.34  & 0.36 &  0.75  &  0.74  &  0.59  \\
 5  &   0.62   & 0.59  & 0.42  & 0.66 &  0.61  &  0.62  &  0.43  &   0.73   & 0.25  & 0.34  & 0.36 &  0.74  &  0.73  &  0.59  \\
 6  &   0.63   &  0.6  & 0.42  & 0.66 &  0.62  &  0.62  &  0.43  &   0.72   & 0.25  & 0.34  & 0.36 &  0.73  &  0.72  &  0.59  \\
 7  &   0.63   &  0.6  & 0.42  & 0.67 &  0.63 

In [11]:
sgpr_crps.shape

(7, 16)